# Exercise: Nonlinear Programming¶

In [173]:
import os
import numpy as np
from numpy.testing import assert_array_equal, assert_array_almost_equal
import matplotlib.pyplot as plt
from casadi import (
    DM, SX, MX, Function,
    vertcat, horzcat,
    norm_2,
    rootfinder
)

import casadi
casadi.__version__

'3.6.4'

In [174]:
plot_dir = 'plots'
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)

## Golf Ball Problem

\begin{aligned}
\cases{\begin{matrix}
    \dot{\mathbf{p}} = \mathbf{v} \\
    \dot{\mathbf{v}} = \mathbf{g} - \frac{1}{m} c \| \mathbf{v} \|_2 \mathbf{v}
\end{matrix}
}
\end{aligned}

where

\begin{aligned}
\mathbf{p} = \left[ \begin{matrix} x \\ y \end{matrix} \right],
\mathbf{v} = \left[ \begin{matrix} v_x \\ v_y \end{matrix} \right],
\mathbf{g} = \left[ \begin{matrix} 0 \\ -g \end{matrix} \right]
\end{aligned}

## 1. Integrating the golf ball’s state

### 1.1 CasADi expression for the ODE

In [189]:
p = MX.sym('p', 2)
v = MX.sym('v', 2)

# Constants
m = DM(0.04593)
g = SX(2, 1)
g[1] = -9.81
c = DM(17.5e-5)
g = DM(2, 1)
g[1] = -9.81
g

DM([00, -9.81])

In [190]:
rhs = vertcat(
    v,
    g - c * norm_2(v) * v / m
)

assert rhs.shape == (4, 1)
rhs

MX(vertcat(v, ([0, -9.81]-(((0.000175*||v||_F)*v)/0.04593))))

In [194]:
f = Function('rhs', [vertcat(p, v)], [rhs])
assert_array_almost_equal(
    f([0.0, 0.0, 35.0, 30.0]), 
    [[35], [30], [-6.14737], [-15.0792]], 
    decimal=4
)